# BART Tutorial: Subspace-Constrained Reconstruction of Frequency-Modulated SSFP Data

This notebook demonstrates how to use BART to
perform a subspace-constrained reconstruction in the style of

Roeloffs, Volkert, et al. "Frequency‐modulated SSFP with radial sampling and subspace reconstruction: A time‐efficient alternative to phase‐cycled bSSFP." Magnetic resonance in medicine 81.3 (2019): 1566-1579.


Note: Run this notebook with a bash kernel.

Volkert Roeloffs, University Medical Center Göttingen (volkert.roeloffs@med.uni-goettingen.de)

March 2021

# 1. Prerequisites

Make sure these paths are set correctly

`TOOLBOX_PATH=/path/to/bart`

`PATH=$TOOLBOX_PATH:$PATH`

In [ ]:
# check bart availability and version - we recommend the most updated version (v0.7.00)
bart version

In [ ]:
# helper BASH function to write BART file to png,
# display both, magnitude and phase

function imshow () {
    cfl2png -CM $1 magn > /dev/null
    cfl2png -CP $1 phas > /dev/null

    cat magn_.png | display
    cat phas_.png | display
}

function show_timesteps () {

    data=$1
    shift
    pos=("$@")

    ind=0

    for t in "${pos[@]}"
    do
       bart slice 5 $t $data _slice$ind
       ind=$((ind+1))
    done

    bart join 6 `seq -f "_slice%g" 0 $((ind-1))` _slices
    DIM=`bart show -d 0 _slices`
    bart reshape $(bart bitmask 0 6) $((DIM*ind)) 1 {_,}slices

    imshow slices
}

# 2. Raw data

## 2.1 Loading Raw data
In the original publication 3D stack-of-stars encoded data sets are reconstructed. For demonstration purposes, we consider here only a single slice as obtained after inverse Fourier transforming along partition encoding direction.

In [ ]:
wget -q https://zenodo.org/record/4060287/files/fmSSFP.cfl
wget -q https://zenodo.org/record/4060287/files/fmSSFP.hdr

bart show -m fmSSFP
PHS1=`bart show -d 1 fmSSFP`
SPOKES=`bart show -d 2 fmSSFP`
COILS=`bart show -d 3 fmSSFP`

## 2.2 Visualizing raw data

In [ ]:
bart reshape `bart bitmask 1 3` $((PHS1*COILS)) 1 fmSSFP tmp
imshow tmp

# 3. Reconstruction in Subspace

## 3.1 Generation of low-frequency Fourier basis

In [ ]:
P=4 # subspace size

bart delta 16 $(bart bitmask 5 6) $P eye
bart crop 6 $P eye eye_c 
bart resize -c 5 $SPOKES eye_c eye_cl
bart fft -u $(bart bitmask 5) eye_cl basis

## 3.2 Subspace-constrained reconstruction

### 3.2.1 Generate radial, turn-based trajectory corrected for gradient delays

In [ ]:
SPT=$((SPOKES/P)) # spokes per turn

GD="0.114442:0.058800:-0.068824"
# gradient delays were determined with RING (Rosenzweig et al., 2018) 
# on the DC partition of the full 3D stack-of-stars data 

bart traj -q$GD -r -c -D -x$PHS1 -y$SPT traj_st_1
bart repmat 3 $P traj_st_1 traj_f_1
bart reshape $(bart bitmask 2 3) $SPOKES 1 traj_f_1 traj_1

### 3.2.2 Channel compression, inverse nuFFT and ESPIRIT

In [ ]:
VCOILS=10

# perform channel compression on combined data
bart cc -p$VCOILS -A -S fmSSFP meas_cc
    
# apply inverse nufft 
bart nufft -i -d$PHS1:$PHS1:1 traj_1 meas_cc img
    
# transform back to k-space and compute sensitivities
bart fft -u $(bart bitmask 0 1 2) img ksp

# transpose because we already support off-center calibration region
# in dim 0 but here we might have it in 2
bart transpose 0 2 ksp ksp2
bart ecalib -S -t0.01 -m1 ksp2 sens2
bart transpose 0 2 sens2 sens

### 3.2.3 Bringing it all together: Use basis and sensitivities for subspace-constrained reconstruction

In [ ]:
REG=0.002
LLRBLK=8
ITERATIONS=100

# transform data
bart transpose 2 5 meas_cc meas_t
bart transpose 2 5 traj_1 traj_t

# reconstruction with subspace constraint
bart pics -SeH -d5 -R L:3:3:$REG -i$ITERATIONS -t traj_t -B basis meas_t sens out

# flags:
#        -S rescale image after reconstruction
#        -e scale stepsize based on maximal eigenvalue
#        -H hidden option "hogwild", allowing asynchronous updates 
#        -R L:3:3 

In [ ]:
# crop, transpose, flip to resize FoV and get anatomically correct orientation
READ_NOOS=$((PHS1/2))
bart crop 0 $READ_NOOS out out_c
bart crop 1 $READ_NOOS out_c out_cc
bart transpose 0 1 out_cc _tmp
bart flip `bart bitmask 0` _tmp __tmp

bart reshape `bart bitmask 0 6` $((READ_NOOS*4)) 1 __tmp tmp
imshow tmp

In [ ]:
# generate composite image by combining subspace coefficients in root-of-sum-of-squares manner
bart rss `bart bitmask 6` __tmp rss_image
imshow rss_image

In [ ]:
# synthesize phase-cycled bSSFP-like images (time domain projection)
bart fmac -s `bart bitmask 6` __tmp basis bSSFPsyn
show_timesteps bSSFPsyn 0 101 202 303